# Script to transform the EXCEL-Sheet with the transition-Matrix into a valid json-formatted file which can be placed into the flow

# Make imports

In [1]:
import json
import pandas as pd

# Define variables

In [2]:
source_file = './valves_matrix_edited.xlsx'
source_table = 'Transition_Matrix'
destination_file = source_file.replace('.xlsx','.json')

# Read the excel sheet and show that it worked

In [3]:
df = pd.read_excel(source_file,sheet_name=source_table)
df.head()

,STATE,EVENT,ACTION,NEW_STATE
0,CLOSED,ack_fault_recieved,*,-
1,CLOSED,command_close_recieved,*,-
2,CLOSED,signal_closed_recieved,*,-
3,CLOSED,timer_up_closing,*,-
4,CLOSED,timer_up_opening,*,-


# Example how the flow is formatted

In [4]:
example = '''
[
    {
        "id": "17fbbe06.3b0bfa",
        "type": "FSM",
        "z": "1c3e6ba1.a860d4",
        "name": "test",
        "topic": "test",
        "initstate": "test",
        "inittran": "TR_INIT",
        "entry": false,
        "exit": false,
        "tran": true,
        "rules": [
            {
                "n": "INIT__cmd_open__OPENING__act_start_opening_timer",
                "c": "cmd_open",
                "s": "INIT",
                "d": "OPENING",
                "t": "true"
            }
        ],
        "x": 480,
        "y": 220,
        "wires": [
            []
        ]
    }
]
'''

# Generate a list of rules which follow the formatting-rule of the flow

In [5]:
separator = '__'
rules = []
for idx, row in df.iterrows():
    #Unpack the row
    state = row['STATE']
    event = row['EVENT']
    action = row['ACTION']
    action_str = ';'.join(action.split(', '))
    new_state = row['NEW_STATE']
    
    #Check if the entry does anything
    if not action_str == '*' or not new_state == '-':
        # If no transition but action required, transition back into current state to trigger a msg-object
        if new_state == '-':
            new_state = state
        new_rule = {
            "n": state+separator+event+separator+new_state+separator+action_str,
            "c": event,
            "s": state,
            "d": new_state,
            "t": "true",
        }
        rules.append(new_rule)

# Convert the list to json-syntax and save it to file

In [6]:
json_str = json.dumps(rules)
with open(destination_file,'w') as f:
    f.write(json_str)
print('All Done')

All Done
